In [1]:
import json
import openai
import yfinance as yf
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
client = openai.Client()

In [2]:
ticker = 'ABEV3'
ticker_obj = yf.Ticker(f'{ticker}.SA')
print(ticker_obj)

period = '5d'
ticker_obj.history(period=period)

yfinance.Ticker object <ABEV3.SA>


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-03-05 00:00:00-03:00,12.50,12.93,12.42,12.78,54086900,0.0,0.0
2025-03-06 00:00:00-03:00,12.77,13.04,12.70,12.85,40524500,0.0,0.0
2025-03-07 00:00:00-03:00,12.78,13.12,12.72,13.10,30190400,0.0,0.0
2025-03-10 00:00:00-03:00,13.00,13.26,12.98,13.12,27718600,0.0,0.0
2025-03-11 00:00:00-03:00,13.08,13.16,12.74,12.86,32659900,0.0,0.0


In [3]:
def retorna_cotacao_acao_historica(
    ticker,
    periodo='1mo'
):
    ticker_obj = yf.Ticker(f'{ticker}.SA')
    hist = ticker_obj.history(period=periodo)
    hist.index = hist.index.strftime('%Y-%m-%d')
    hist = round(hist, 2)
    if len(hist) > 30:
        slice_size = int(len(hist)/30)
        hist = hist.iloc[::-slice_size][::-1]
    return hist['Close'].to_json()
hist = retorna_cotacao_acao_historica('ABEV3','5y')
hist

'{"2020-03-30":10.02,"2020-05-29":9.96,"2020-07-28":12.05,"2020-09-24":10.12,"2020-11-25":11.57,"2021-01-28":12.8,"2021-03-30":12.84,"2021-05-28":14.7,"2021-07-28":14.27,"2021-09-24":13.04,"2021-11-25":14.19,"2022-01-25":12.89,"2022-03-25":12.25,"2022-05-25":12.01,"2022-07-22":12.41,"2022-09-20":13.48,"2022-11-21":13.41,"2023-01-18":12.41,"2023-03-20":12.72,"2023-05-19":13.13,"2023-07-18":13.21,"2023-09-14":12.19,"2023-11-14":12.34,"2024-01-16":12.65,"2024-03-15":11.97,"2024-05-15":11.67,"2024-07-12":11.03,"2024-09-09":12.43,"2024-11-05":12.02,"2025-01-09":11.4,"2025-03-11":12.86}'

In [4]:
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'retorna_cotacao_acao_historica',
            'description': 'Retorna a cotação diária histórica de uma ação da bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type': 'string',
                        'description': 'O ticker do ativo. Exemplo: ABVE3 para ambev, PETR4 para petrobras, etc'
                    },
                    'periodo': {
                        'type': 'string',
                        'description': 'O período que será retornado de dados históricos \
                                        sendo "1mo" equivalente a um mês de dados, "1d" a \
                                        1 dia e "1y" a 1 ano',
                        'enum': ['1d','5d','1mo','3mo','6mo','1y','2y','5y','10y','ytd','max']
                    }
                }
            }
        }
    }
]

funcoes_disponiveis = {'retorna_cotacao_acao_historica': retorna_cotacao_acao_historica}

In [9]:
mensagens = [{'role': 'user', 'content': 'Qual é a cotação da ambev agora?'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    tools=tools,
    tool_choice='auto'
)

In [ ]:
tool_calls = resposta.choices[0].message.tool_calls

if tool_calls:
    mensagens.append(resposta.choices[0].message)
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[func_name]
        func_args = json.loads(tool_call.function.arguments)
        func_return = function_to_call(**func_args)
        mensagens.append({
            'tool_call_id': tool_call.id,
            'role': 'tool',
            'name': func_name,
            'content': func_return
        })
    segunda_resposta = client.chat.completions.create(
        messages=mensagens,
        model='gpt-3.5-turbo-0125'
    )
    mensagens.append(segunda_resposta.choices[0].message)

'A cotação atual da Ambev é de R$ 12,86.'